In [24]:
import requests
import csv
import json

# Overpass API 查询（新泽西州牙科诊所）
overpass_query = """
[out:json][timeout:30];
area["name"="New Jersey"]->.searchArea;
(
  node["amenity"="dentist"](area.searchArea);
  way["amenity"="dentist"](area.searchArea);
  relation["amenity"="dentist"](area.searchArea);
);
out body;
>;
out skel qt;
"""

# 请求 Overpass API
url = "https://overpass-api.de/api/interpreter"
response = requests.post(url, data=overpass_query)
data = response.json()

# 提取并清洗数据
clinics = []
for element in data["elements"]:
    if "tags" not in element:
        continue
    
    tags = element["tags"]
    
    # 诊所名称（必填）
    name = tags.get("name", "Unknown Dental Clinic")
    
    # 直接获取已标注的地址（优先使用）
    address = tags.get("addr:full", "")
    if not address:
        # 若未标注完整地址，拼接各部分
        address_parts = [
            tags.get("addr:street", ""),
            tags.get("addr:city", ""),
            tags.get("addr:state", "NJ"),
            tags.get("addr:postcode", "")
        ]
        address = ", ".join(filter(None, address_parts))  # 过滤空值
    
    # 若地址仍为空，尝试通过经纬度反向解析（需联网）
    if not address and ("lat" in element and "lon" in element):
        try:
            lat, lon = element["lat"], element["lon"]
            nominatim_url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
            nominatim_response = requests.get(nominatim_url, headers={"User-Agent": "Mozilla/5.0"})
            if nominatim_response.status_code == 200:
                nominatim_data = nominatim_response.json()
                address = nominatim_data.get("display_name", "")
        except Exception as e:
            print(f"反向地理编码失败：{e}")
    
    # 电话（保留原始数据，不添加 +1）
    phone = tags.get("phone", "").strip()
    
    # 邮箱和网站
    email = tags.get("email", "")
    website = tags.get("website", "")
    
    clinics.append({
        "name": name,
        "address": address if address else "",  # 确保无地址时留空
        "phone": phone if phone else "",
        "email": email if email else "",
        "website": website if website else ""
    })

# 导出为 CSV（仅保留名称、地址、电话、邮箱、网站）
csv_columns = ["name", "address", "phone", "email", "website"]
csv_file = "new_jersey_dentists_final.csv"

with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(clinics)

print(f"成功导出 {len(clinics)} 家牙科诊所数据到 {csv_file}")

成功导出 523 家牙科诊所数据到 new_jersey_dentists_final.csv
